<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir=rtl>
<h3><center>تمرین سوم درس پردازش زبان‌های طبیعی</center></h3>
<h4><center>چالش جست‌وجوی دارو</center></h4>
<table width='100%' style="border: none;">
    <tr style="border: none; text-align: center;">
        <td style="border: none;"><h5>علیرضا بلال</h5></td>
        <td style="border: none;"><h5>زهرا رجالی</h5></td>
        <td style="border: none;"><h5>جواد راضی</h5></td>
    </tr>
        <tr style="border: none; text-align: center;">
        <td style="border: none;"><h5>400200881</h5></td>
        <td style="border: none;"><h5>401204716</h5></td>
        <td style="border: none;"><h5>401204354</h5></td>
    </tr>
</table>
<h5 style="font-size: 16px;"><center> بهار ۱۴۰۲ </center></h5>
<br/>
<hr/>
<br/>

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar', 'B Lotus', 'Calibri'" size=3><div dir='rtl' align='justify'>
<b>
    فایل ژوپیتر این تمرین در کولب توسعه داده و تست شده‌است. این فایل هم در محیط کولب، هم با ایمیج داکر jupyter/datascience-notebook تست شده‌است و همه قطعه‌کدها خروجی مورد انتظار را می‌دهند. اگه در بازتولید خروجی بعضی سل‌ها، یا کدهای تمرین مشکلی وجود داشت، ممنون می‌شویم در صورت امکان به ما اطلاع دهید تا فایل را در محیطی که قابل اجرا است، اجرا نموده و خروجی را نمایش دهیم. 
</b>

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

### واردکردن و نصب کتاب‌خانه‌های مورد استفاده


In [26]:
try:
    import ipywidgets
except:
    %pip install ipywidgets 
    
try: 
    import tensorflow as tf
except:
   %pip install tensorflow

try: 
    import keras_nlp
except: 
    %pip install keras-nlp

try:
    import torch
except:
    %pip install torch torchvision torchaudio
    
try: 
    import pandas as pd
except:
    %pip install pandas 

try: 
    import bert
except: 
    %pip install bert 
    
try: 
    import nltk
except: 
    %pip install nltk 
    

try:
    import gensim 
except:
    %pip install gensim 

try: 
    import langdetect
except: 
    %pip install langdetect 
    
try:
    import matplotlib as mpl
except:
    %pip install matplotlib 
    
try:
    import sklearn
except:
    %pip install sklearn
    
try:
    import numpy as np
except:
    %pip install numpy

In [27]:
# Some configuration for pandas for a better display of tables
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("max_colwidth", None)

In [28]:
from IPython.display import HTML

# Set the font family and fallback fonts for pandas output globally

mpl.rcParams['font.family'] = ['vazirmatn', 'Vazir', 'B Nazanin', 'Arial']
mpl.rcParams['font.sans-serif'] = ['vazirmatn', 'Vazir', 'B Nazanin', 'Arial']
mpl.rcParams['font.serif'] = ['vazirmatn', 'Vazir', 'B Nazanin', 'Arial']

def set_pandas_font(fonts):
    css = f"""
    <style>
        table.dataframe td, table.dataframe th {{
            font-family: {fonts};
        }}
    </style>
    """
    return HTML(css)

set_pandas_font("'vazirmatn', 'Vazir', 'B Nazanin', 'Arial'")

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#    **خلاصه‌ای از صورت مسئله و راه‌حل**
اینجا یه خلاصه می‌دیم که مسئله چیه. بعد یه outline از کارایی که کردیم می‌دیم. 

کارایی که کردیم (حالا به شکل بهتری بیان میشن):‌
۱- یافتن دیتابیسی از داروهای انگلیسی
۲- نرمالیزه کردن دیتابیس به شکلی که می‌خوایم 
۳- یافتن معادل فارسی داروهای دیتابیس 


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#    **مراحل طراحی سامانه جستجوی دارو**
در این بخش، مراحلی که برای طراحی سامانه جستجوی دارو طی شده‌اند و در قسمت پیشین خلاصه‌ای از آن بیان شد، به تفصیل شرح داده خواهد شد. 



<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

## ۱. یافتن دیتابیس داروها به زبان انگلیسی

در وب‌سایت سازمان غذا و داروی ایران، دیتاستی با حدود ۵۰ هزار ورودی که برای هر دارو شامل نام دارو به فارسی و لاتین، نام جنریک آن، نام برند و موارد دیگر می‌باشد، برای دانلود در دسترس عام است. دیتابیس اصلی استفاده شده در این تمرین، این دیتابیس می‌باشد. البته بر روی این دیتابیس، تغییرات و نرمال‌سازی‌هایی انجام شده‌است که در ادامه ذکر خواهند شد. 
دیتابیس دیگر، از این مخزن گیت‌هاب دانلود شد. این دیتابیس، شامل نام هر دارو به ازای کد ATC آن می‌باشد. کد ATC دارو، که در دیتاست سازمان غذا و داروی ایران نیز موجود است، طبقه‌بندی داروها رو توصیف می‌کند. برای مثال، برای داروی ملاتونین که یک قرص تنظیم ساعت خواب است، کد ATC آن N05CH01 می‌باشد. در این کد، کاراکترها به ترتیب، سلسله‌مراتب دسته‌ای که دارو به آن متعلق است را مشخص می‌کنند. برای داروی ذکرشده، این سلسله مراتب به شرح زیر است:
- N: سیستم عصبی
- N05: داروهای روان‌پزشکی
- N05C: خواب‌آور و آرام‌بخش
- N05CH: آگونیست گیرنده‌های ملاتونین
- N05CH01: قرص ملاتونین ۲ میلی‌گرم

همان‌طور که از این مثال پیداست، کد ATC دارو اطلاعات ارزشمندی در خصوص دسته‌بندی، و کارکرد آن به ما می‌دهد. با توجه به اینکه دیتاست اصلی سازمان غذا و دارو، برای هر دارو کد ATC آن را نیز دارد، با Join نمودن این دو دیتاست، توانستیم برای هر دارو، دسته‌بندی و کاربردهایی از آن را در قالب چند ستون اضافه، به دیتاست اضافه نماییم. 

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

## ۲. نرمال‌سازی و پیش‌پردازش‌های روی دیتابیس

### حذف داده‌های اضافه
دیتابیس دارهای سازمان غذا و دارو، دارای ستون‌هایی بود که برای این تمرین، کاربردی نداشتند. این ستون‌ها، شامل ستون‌هایی نظیر آمار فروش و قیمت داروها، و امکان فروش بدون نسخه داروها بودند. از آن‌جایی که این‌کار پیچیدگی خاصی نداشت، ستون‌های اضافه به صورت دستی از فایل حذف شدند. 



<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

### نرمال‌سازی داده‌ها
برخی نرمال‌سازی‌ها بر روی دیتاست داروها انجام گردید. مقدار تمام مدخل‌های جدول، به فرم lowercase آن تبدیل شد.  

In [29]:
IR_FDA_DATASET_PATH = "./data/Iran_FDA_1400_Dataset.csv"
WHO_ATC_INDEX_PATH = "./data/WHO_ATC_Index.csv"

medicine_ds = pd.read_csv(IR_FDA_DATASET_PATH)

medicine_ds.head()

,pharma_company,supplier,brand_owner,distributor,manufacturer_country,brand_name_fa,brand_name_en,generic_name_en,substance_name_en,atc_code
0,Actero middleeast,اکتوورکو,اکتوورکو,الیت دارو,ایران,فاویپیراویر قرص خوراکی 200 mg,FAVIPIRAVIR TABLET ORAL 200 mg,FAVIPIRAVIR TABLET ORAL 200 mg,FAVIPIRAVIR,J05AX
1,Actoverco,اکتوورکو,اکتوورکو,الیت دارو,ایران,رمدسیویر محلول تزریقی پرنترال 5 mg/1mL 20 mg,"REMDESIVIR INJECTION, SOLUTION PARENTERAL 5 mg/1mL 20 mg","REMDESIVIR INJECTION, SOLUTION PARENTERAL 5 mg/1mL 20 mL",REMDESIVIR,J05
2,Actoverco,اکتوورکو,"Krka, D. D., Novo Mesto",الیت دارو,ایران,نولپازا قرص انتریک کوتد خوراکی 40 mg,"NOLPAZA TABLET, DELAYED RELEASE ORAL 40 mg","PANTOPRAZOLE TABLET, DELAYED RELEASE ORAL 40 mg",PANTOPRAZOLE,A02BC02
3,Actoverco,اکتوورکو,"Krka, D. D., Novo Mesto",الیت دارو,ایران,آسنترا قرص خوراکی 50 mg,ASENTRA TABLET ORAL 50 mg,SERTRALINE (AS HYDROCHLORIDE) TABLET ORAL 50 mg,SERTRALINE (AS HYDROCHLORIDE),N06AB06
4,Actoverco,اکتوورکو,اکتوورکو,الیت دارو,ایران,ناکسپرین تزریقی پرنترال 100 mg/1mL 0.4 mL,NOXPRIN INJECTION PARENTERAL 100 mg/1mL 0.4 mL,ENOXAPARIN SODIUM INJECTION PARENTERAL 100 mg/1mL 0.4 mL,ENOXAPARIN SODIUM,B01AB05


In [30]:
atc_index_ds = pd.read_csv(WHO_ATC_INDEX_PATH)

atc_index_ds.head()

,atc_code,atc_name,ddd,uom,adm_r,note
0,A,ALIMENTARY TRACT AND METABOLISM,NaN,NaN,NaN,NaN
1,A01,STOMATOLOGICAL PREPARATIONS,NaN,NaN,NaN,NaN
2,A01A,STOMATOLOGICAL PREPARATIONS,NaN,NaN,NaN,NaN
3,A01AA,Caries prophylactic agents,NaN,NaN,NaN,NaN
4,A01AA01,sodium fluoride,1.1,mg,O,0.5 mg fluoride


In [31]:
import pandas as pd
from langdetect import detect

# Lowercase all entries in the drug dataset
medicine_ds[medicine_ds.select_dtypes(['object']).columns] = medicine_ds.select_dtypes(['object']).apply(lambda x: x.str.lower())

medicine_ds.head()

,pharma_company,supplier,brand_owner,distributor,manufacturer_country,brand_name_fa,brand_name_en,generic_name_en,substance_name_en,atc_code
0,actero middleeast,اکتوورکو,اکتوورکو,الیت دارو,ایران,فاویپیراویر قرص خوراکی 200 mg,favipiravir tablet oral 200 mg,favipiravir tablet oral 200 mg,favipiravir,j05ax
1,actoverco,اکتوورکو,اکتوورکو,الیت دارو,ایران,رمدسیویر محلول تزریقی پرنترال 5 mg/1ml 20 mg,"remdesivir injection, solution parenteral 5 mg/1ml 20 mg","remdesivir injection, solution parenteral 5 mg/1ml 20 ml",remdesivir,j05
2,actoverco,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,نولپازا قرص انتریک کوتد خوراکی 40 mg,"nolpaza tablet, delayed release oral 40 mg","pantoprazole tablet, delayed release oral 40 mg",pantoprazole,a02bc02
3,actoverco,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,آسنترا قرص خوراکی 50 mg,asentra tablet oral 50 mg,sertraline (as hydrochloride) tablet oral 50 mg,sertraline (as hydrochloride),n06ab06
4,actoverco,اکتوورکو,اکتوورکو,الیت دارو,ایران,ناکسپرین تزریقی پرنترال 100 mg/1ml 0.4 ml,noxprin injection parenteral 100 mg/1ml 0.4 ml,enoxaparin sodium injection parenteral 100 mg/1ml 0.4 ml,enoxaparin sodium,b01ab05


In [32]:
# Lowercase all entries in the ATC index dataset
atc_index_ds[atc_index_ds.select_dtypes(['object']).columns] = atc_index_ds.select_dtypes(['object']).apply(lambda x: x.str.lower())

atc_index_ds.head()

,atc_code,atc_name,ddd,uom,adm_r,note
0,a,alimentary tract and metabolism,NaN,NaN,NaN,NaN
1,a01,stomatological preparations,NaN,NaN,NaN,NaN
2,a01a,stomatological preparations,NaN,NaN,NaN,NaN
3,a01aa,caries prophylactic agents,NaN,NaN,NaN,NaN
4,a01aa01,sodium fluoride,1.1,mg,o,0.5 mg fluoride


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

### جداسازی تکه‌های معنادار کد ATC داروها
همان‌طور که در بالا گفته شد، کد ATC داروها شامل قسمت‌های معناداری است که به صورت سلسله‌مراتبی، طبقه‌بندی دارو،‌و حوزه استفاده آن را مشخص می‌کند. در این قسمت، کد ATC هر دارو، مطابق مستنداتی که ساختار کد ATC را شرح داده‌اند، به ۵ قسمت تقسیم شده و این ۵ قسمت، در قالب ۵ ستون (ویژگی) به دیتاست اصلی اضافه شدند. در زیر، کد مربوط به تقسیم شناسه ATC داروها پیاده‌سازی شده‌است. 


In [33]:
def extract_atc_parts(atc_code):
    atc_code = str(atc_code)
    return [atc_code[:1], atc_code[:3] if len(atc_code) >= 3 else None,
            atc_code[:4] if len(atc_code) >= 4 else None, atc_code[:5] if len(atc_code) >= 5 else None,
            atc_code if len(atc_code) >= 7 else None]
 
ATC_CATEGORIES = ['anatomical', 'therapeutic', 'pharmacological', 'subpharmacological', 'chemical']
medicine_atc_splitted = medicine_ds['atc_code'].apply(extract_atc_parts).to_list()

medicine_atc_columns = pd.DataFrame(medicine_atc_splitted, dtype='object', columns=ATC_CATEGORIES)

medicine_ds = medicine_ds.join(medicine_atc_columns, how='inner')
medicine_ds.rename(columns={'atc_code': 'atc_code_raw'}, inplace=True)

medicine_ds.head()

,pharma_company,supplier,brand_owner,distributor,manufacturer_country,brand_name_fa,brand_name_en,generic_name_en,substance_name_en,atc_code_raw,anatomical,therapeutic,pharmacological,subpharmacological,chemical
0,actero middleeast,اکتوورکو,اکتوورکو,الیت دارو,ایران,فاویپیراویر قرص خوراکی 200 mg,favipiravir tablet oral 200 mg,favipiravir tablet oral 200 mg,favipiravir,j05ax,j,j05,j05a,j05ax,None
1,actoverco,اکتوورکو,اکتوورکو,الیت دارو,ایران,رمدسیویر محلول تزریقی پرنترال 5 mg/1ml 20 mg,"remdesivir injection, solution parenteral 5 mg/1ml 20 mg","remdesivir injection, solution parenteral 5 mg/1ml 20 ml",remdesivir,j05,j,j05,None,None,None
2,actoverco,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,نولپازا قرص انتریک کوتد خوراکی 40 mg,"nolpaza tablet, delayed release oral 40 mg","pantoprazole tablet, delayed release oral 40 mg",pantoprazole,a02bc02,a,a02,a02b,a02bc,a02bc02
3,actoverco,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,آسنترا قرص خوراکی 50 mg,asentra tablet oral 50 mg,sertraline (as hydrochloride) tablet oral 50 mg,sertraline (as hydrochloride),n06ab06,n,n06,n06a,n06ab,n06ab06
4,actoverco,اکتوورکو,اکتوورکو,الیت دارو,ایران,ناکسپرین تزریقی پرنترال 100 mg/1ml 0.4 ml,noxprin injection parenteral 100 mg/1ml 0.4 ml,enoxaparin sodium injection parenteral 100 mg/1ml 0.4 ml,enoxaparin sodium,b01ab05,b,b01,b01a,b01ab,b01ab05


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
## ۳. ترکیب دیتاست داروها با دیتاست طبقه‌بندی کد ATC داروها
اینجا توضیح میدیم دو تا دیتاست چجوری جوین شدن

In [34]:
import pandas as pd

atc_columns = ['anatomical', 'therapeutic', 'pharmacological', 'subpharmacological', 'chemical']
desc_columns = ['anatomical_desc', 'therapeutic_desc', 'pharmacological_desc', 'subpharmacological_desc', 'chemical_desc']

medicine_ds_merged = medicine_ds.copy()

for atc_col, desc_col in zip(atc_columns, desc_columns):
    medicine_ds_merged = medicine_ds_merged.merge(atc_index_ds[['atc_code', 'atc_name']], left_on=atc_col, right_on='atc_code', how='left')
    medicine_ds_merged = medicine_ds_merged.rename(columns={'atc_name': desc_col})
    medicine_ds_merged = medicine_ds_merged.drop(columns=['atc_code'])

medicine_ds_merged = medicine_ds_merged.drop(columns=atc_columns)
medicine_ds_merged.head()

,pharma_company,supplier,brand_owner,distributor,manufacturer_country,brand_name_fa,brand_name_en,generic_name_en,substance_name_en,atc_code_raw,anatomical_desc,therapeutic_desc,pharmacological_desc,subpharmacological_desc,chemical_desc
0,actero middleeast,اکتوورکو,اکتوورکو,الیت دارو,ایران,فاویپیراویر قرص خوراکی 200 mg,favipiravir tablet oral 200 mg,favipiravir tablet oral 200 mg,favipiravir,j05ax,antiinfectives for systemic use,antivirals for systemic use,direct acting antivirals,other antivirals,NaN
1,actoverco,اکتوورکو,اکتوورکو,الیت دارو,ایران,رمدسیویر محلول تزریقی پرنترال 5 mg/1ml 20 mg,"remdesivir injection, solution parenteral 5 mg/1ml 20 mg","remdesivir injection, solution parenteral 5 mg/1ml 20 ml",remdesivir,j05,antiinfectives for systemic use,antivirals for systemic use,NaN,NaN,NaN
2,actoverco,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,نولپازا قرص انتریک کوتد خوراکی 40 mg,"nolpaza tablet, delayed release oral 40 mg","pantoprazole tablet, delayed release oral 40 mg",pantoprazole,a02bc02,alimentary tract and metabolism,drugs for acid related disorders,drugs for peptic ulcer and gastro-oesophageal reflux disease (gord),proton pump inhibitors,pantoprazole
3,actoverco,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,نولپازا قرص انتریک کوتد خوراکی 40 mg,"nolpaza tablet, delayed release oral 40 mg","pantoprazole tablet, delayed release oral 40 mg",pantoprazole,a02bc02,alimentary tract and metabolism,drugs for acid related disorders,drugs for peptic ulcer and gastro-oesophageal reflux disease (gord),proton pump inhibitors,pantoprazole
4,actoverco,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,آسنترا قرص خوراکی 50 mg,asentra tablet oral 50 mg,sertraline (as hydrochloride) tablet oral 50 mg,sertraline (as hydrochloride),n06ab06,nervous system,psychoanaleptics,antidepressants,selective serotonin reuptake inhibitors,sertraline


In [35]:
medicine_ds_merged.to_csv("./data/medicine_ds_merged.csv", index=False)

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
## ۴. جداسازی دیتابیس‌های فارسی و انگلیسی
تا این مرحله، دیتاستی که داریم،‌شامل ستون‌هایی است که یا به زبان انگلیسی نوشته‌شده‌اند، یا به زبان فارسی. برای بیشتر ستون‌هایی که داده‌های آن‌ها در یک زبان است، ستون دیگری که معادل داده‌ها در زبان دیگر را داشته‌باشد وجود ندارد. در بخش‌های بعد توضیح داده‌خواهد شد که چگونه با تکنیک‌های Transfer Learning، دیتای فارسی که ناکامل بود، با استفاده از مدل ترین‌شده روی دیتای انگلیسی، ترین شد تا قدرتی معادل مدل انگلیسی داشته‌باشد. 

اما در این بخش، به عنوان نخستین گام، تکه‌های فارسی انگلیسی دیتاست از یکدیگر جدا می‌شوند. به این معنا که ستون‌هایی که فارسی هستند در یک دیتافریم، و ستون‌های انگلیسی در دیتافریم دیگر ذخیره شده، و هر یک از آن‌ها در یک فایل csv جداگانه ذخیره می‌گردند. 

In [36]:
persian_columns = ['supplier', 'brand_owner', 'distributor', 'manufacturer_country', 'brand_name_fa']
english_columns = medicine_ds_merged.columns.difference(persian_columns)
# Select the Persian columns for persian_df
persian_df = medicine_ds_merged[persian_columns]

# Select the remaining columns for english_df
english_df = medicine_ds_merged[english_columns]


In [37]:
persian_df.head()

,supplier,brand_owner,distributor,manufacturer_country,brand_name_fa
0,اکتوورکو,اکتوورکو,الیت دارو,ایران,فاویپیراویر قرص خوراکی 200 mg
1,اکتوورکو,اکتوورکو,الیت دارو,ایران,رمدسیویر محلول تزریقی پرنترال 5 mg/1ml 20 mg
2,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,نولپازا قرص انتریک کوتد خوراکی 40 mg
3,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,نولپازا قرص انتریک کوتد خوراکی 40 mg
4,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,آسنترا قرص خوراکی 50 mg


In [38]:
english_df.head()


,anatomical_desc,atc_code_raw,brand_name_en,chemical_desc,generic_name_en,pharma_company,pharmacological_desc,subpharmacological_desc,substance_name_en,therapeutic_desc
0,antiinfectives for systemic use,j05ax,favipiravir tablet oral 200 mg,NaN,favipiravir tablet oral 200 mg,actero middleeast,direct acting antivirals,other antivirals,favipiravir,antivirals for systemic use
1,antiinfectives for systemic use,j05,"remdesivir injection, solution parenteral 5 mg/1ml 20 mg",NaN,"remdesivir injection, solution parenteral 5 mg/1ml 20 ml",actoverco,NaN,NaN,remdesivir,antivirals for systemic use
2,alimentary tract and metabolism,a02bc02,"nolpaza tablet, delayed release oral 40 mg",pantoprazole,"pantoprazole tablet, delayed release oral 40 mg",actoverco,drugs for peptic ulcer and gastro-oesophageal reflux disease (gord),proton pump inhibitors,pantoprazole,drugs for acid related disorders
3,alimentary tract and metabolism,a02bc02,"nolpaza tablet, delayed release oral 40 mg",pantoprazole,"pantoprazole tablet, delayed release oral 40 mg",actoverco,drugs for peptic ulcer and gastro-oesophageal reflux disease (gord),proton pump inhibitors,pantoprazole,drugs for acid related disorders
4,nervous system,n06ab06,asentra tablet oral 50 mg,sertraline,sertraline (as hydrochloride) tablet oral 50 mg,actoverco,antidepressants,selective serotonin reuptake inhibitors,sertraline (as hydrochloride),psychoanaleptics


In [39]:
persian_df.to_csv('./data/persian_df.csv', index=False)
english_df.to_csv('./data/english_df.csv', index=False)

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
## ۵. آموزش مدل‌های زبانی
توضیحات در خصوص اینکه دو تا مدل زبانی آموزش داده شده‌اند و با Transfer Learning،‌کاستی‌های دیتاست فارسی جبران شده. 

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
### ۵.۱. آموزش مدل روی دیتاست انگلیسی
توضیحات در خصوص نحوه آموزش روی دیتاست انگلیسی

In [40]:
import pandas as pd
from gensim.models import Word2Vec

# Load the dataset
data = pd.read_csv('./data/english_df.csv')

sentences = data.apply(lambda x: ' '.join(x.dropna()), axis=1).apply(lambda x: x.split())

# Train the skip-gram model
model = Word2Vec(sentences, min_count=1, sg=1)

# Save the model
model.save('./model/en_skipgram.model')


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
### ۵.۱. آموزش مدل روی دیتاست فارسی
توضیحات در خصوص آموزش روی دیتاست فارسی، و استفاده از تکنیک ترنسفرلرنینگ برای جبران کاستی‌های دیتاست فارسی

In [41]:
import pandas as pd
from gensim.models import Word2Vec

# Load the pre-trained English model
model = Word2Vec.load('./model/en_skipgram.model')

# Preprocess the text data
sentences = persian_df.apply(lambda x: ' '.join(x.dropna()), axis=1).apply(lambda x: x.split())

# Fine-tune the model on the Persian dataset
model.build_vocab(sentences, update=True)
model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)

# Save the fine-tuned model
model.save('./model/fa_skipgram.model')


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
## ۶. مشترک‌سازی فضای معنی بین امبدینگ‌های دو زبان 
توضیح در مورد اینکه چجوری با چه ابزاری، دو تا مدل جدید تولید شدن که امدبینگ‌ها رو به صورت aligned دارن. 

In [43]:
from gensim.models import KeyedVectors
from sklearn.cross_decomposition import CCA

# Load the pre-trained models
english_model = KeyedVectors.load('./model/en_skipgram.model')
persian_model = KeyedVectors.load('./model/fa_skipgram.model')

# Get the common vocabulary
# TODO : Check if there is a better way than to get the common vocabulary
common_vocab = list(set(english_model.wv.index_to_key) & set(persian_model.wv.index_to_key))

# Get the embeddings for the common vocabulary
english_embeddings = english_model.wv[common_vocab]
persian_embeddings = persian_model.wv[common_vocab]


# Align the embeddings using CCA
cca = CCA(n_components=english_embeddings.shape[1], max_iter=1000, tol=1e-4)
english_aligned, persian_aligned = cca.fit_transform(english_embeddings, persian_embeddings)

# Save the aligned embeddings
english_model.wv.vectors = english_aligned
persian_model.wv.vectors = persian_aligned

# Update the English model with the aligned embeddings
english_model = KeyedVectors(vector_size=english_aligned.shape[1])
english_model.add_vectors(common_vocab, english_aligned)
english_model.save('./model/english_aligned.model')

# Update the Persian model with the aligned embeddings
persian_model = KeyedVectors(vector_size=persian_aligned.shape[1])
persian_model.add_vectors(common_vocab, persian_aligned)
persian_model.save('./model/persian_aligned.model')


c:\Users\Javad\mambaforge\envs\nlp\lib\site-packages\sklearn\cross_decomposition\_pls.py:110: ConvergenceWarning: Maximum number of iterations reached
  warnings.warn("Maximum number of iterations reached", ConvergenceWarning)


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
## ۷. جست‌و‌جوی اطلاعات دارویی با ورودی دو زبانه 
اینجا توضیح داده‌میشه که چجوری اطلاعات دارویی رو که کاربر می‌تونه به انگلیسی، فارسی، یا ترکیبی از این دو تا وارد کنه، با مدلایی که ترین کردیم سرچ می‌کنیم و وقتی دارو پیدا شد، در قالب یک آبجکت JSON اطلاعاتش برگردونده میشن. 

In [50]:
import pandas as pd
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

# Load the aligned models
english_model = KeyedVectors.load('./model/english_aligned.model')
persian_model = KeyedVectors.load('./model/persian_aligned.model')

# Load the drug datasets
english_data = pd.read_csv('./data/english_df.csv')
persian_data = pd.read_csv('./data/persian_df.csv')

# Preprocess the text data
english_sentences = english_data[english_columns].apply(lambda x: ' '.join(x.dropna()), axis=1).apply(lambda x: x.split())
persian_sentences = persian_data[persian_columns].apply(lambda x: ' '.join(x.dropna()), axis=1).apply(lambda x: x.split())

# Compute the average embeddings for each drug
english_embeddings = []
for sentence in english_sentences:
    embeddings = [english_model[word] for word in sentence if english_model.has_index_for(word)]
    if embeddings:
        avg_embedding = np.mean(embeddings, axis=0)
        english_embeddings.append(avg_embedding)
    else:
        english_embeddings.append(np.zeros(english_model.vector_size))
persian_embeddings = []
for sentence in persian_sentences:
    embeddings = [persian_model[word] for word in sentence if persian_model.has_index_for(word)]
    if embeddings:
        avg_embedding = np.mean(embeddings, axis=0)
        persian_embeddings.append(avg_embedding)
    else:
        persian_embeddings.append(np.zeros(persian_model.vector_size))

# Define a function to return drug information based on a user query
def search_drugs(query, k=10):
    # Preprocess the query
    query = query.split()

    # Get the embeddings for the query words
    english_query_embeddings = [english_model[word] for word in query if english_model.has_index_for(word)]
    persian_query_embeddings = [persian_model[word] for word in query if persian_model.has_index_for(word)]

    # Check if the query contains any known words
    if not english_query_embeddings and not persian_query_embeddings:
        return pd.DataFrame()

    # Average the embeddings to get a single embedding for the query
    query_embedding = np.mean(english_query_embeddings + persian_query_embeddings, axis=0)

    # Find the most similar drugs in the English dataset
    english_similarities = cosine_similarity([query_embedding], english_embeddings)[0]
    english_results = english_data.iloc[english_similarities.argsort()[::-1][:k]]

    # Find the most similar drugs in the Persian dataset
    persian_similarities = cosine_similarity([query_embedding], persian_embeddings)[0]
    persian_results = persian_data.iloc[persian_similarities.argsort()[::-1][:k]]

    # Combine and return the results
    return pd.concat([english_results, persian_results])

# Example usage
results = search_drugs('antidepressant')
results 


c:\Users\Javad\mambaforge\envs\nlp\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Javad\mambaforge\envs\nlp\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


ValueError: Expected 2D array, got 1D array instead:
array=[nan].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

## **ارزیابی عملکرد سامانه**
اینجا در خصوص تست عملکرد یه سری اطلاعات داده میشه. بعدش هم یک‌سری ورودی‌های سرچ که شامل ورودی‌های متنوع (به زبان‌های مختلف، ترکیبی از دو زبان، با غلط املایی، جستجو بر اساس نام، کتگوری، کاربردهای دارو و غیره) رو تعریف می‌کنیم و نشون می‌دیم به ازای هر ورودی، مدل چه خروجی‌هایی داشته و عملکردش چجوری بوده. 
